Leung Wai Liu <br>
JPMorgan AI Research <br>
August 8, 2022 <br>
Twitter Relabelling WORD Test Data Processing 

In [1]:
from reading_datasets import read_ud_dataset, reading_tb_ner
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
pos_train = read_ud_dataset(dataset = 'tb', location = '../Datasets/POSTagging/Tweebank/', split = 'train')
pos_val = read_ud_dataset(dataset = 'tb', location = '../Datasets/POSTagging/Tweebank/', split = 'dev')
pos_test = read_ud_dataset(dataset = 'tb', location = '../Datasets/POSTagging/Tweebank/', split = 'test')
ner_train = reading_tb_ner(location = '../Datasets/NER/Tweebank/', split = 'train')
ner_val = reading_tb_ner(location = '../Datasets/NER/Tweebank/', split = 'dev')
ner_test = reading_tb_ner(location = '../Datasets/NER/Tweebank/', split = 'test')

#Getting all the tweets for testing dataset
combined_dataset_test = []
for i, (tweet, pos_labels) in enumerate(pos_test):
    tweet = [token.lower() for token in tweet]
    query = ' '.join(tweet)

    found = False
    for ner_tweet, ner_labels in ner_test:
        ner_tweet = [token.lower() for token in ner_tweet]
        check = ' '.join(ner_tweet)
        if query == check:
            found = True
            break

    if found and tweet == ner_tweet:
        combined_dataset_test.append([tweet, pos_labels, ner_labels])


In [3]:
combined_test_ner_per = []
combined_test_ner_org = []
combined_test_ner_loc = []
combined_test_ner_misc = []
combined_test_ner_o = []

for tweet, pos_labels, ner_labels in combined_dataset_test:
    for t, p, n in zip(tweet, pos_labels, ner_labels):
        if n == "O":
            combined_test_ner_o.append(t)
        else:
            new_n = n[2:]
            if new_n == "PER":
                combined_test_ner_per.append(t)
            elif new_n == "ORG":
                combined_test_ner_org.append(t)
            elif new_n == "LOC":
                combined_test_ner_loc.append(t)
            elif new_n == "MISC":
                combined_test_ner_misc.append(t)


In [4]:
test_ner_per_counter = Counter(combined_test_ner_per)

print(len(test_ner_per_counter))


342


In [5]:
per_counter_df = pd.DataFrame.from_dict(test_ner_per_counter, orient='index').reset_index()
per_counter_df = per_counter_df.rename(columns={'index': 'PER', 0: 'Count'})
per_counter_df = per_counter_df.sort_values(by="Count", ascending=False, ignore_index=True)
per_counter_df

,PER,Count
0,jesus,5
1,trump,4
2,louis,4
3,harry,4
4,donald,4
...,...,...
337,beckham,1
338,david,1
339,trumps,1
340,putin,1


In [6]:
test_ner_org_counter = Counter(combined_test_ner_org)

print(len(test_ner_org_counter))

188


In [7]:
org_counter_df = pd.DataFrame.from_dict(test_ner_org_counter, orient='index').reset_index()
org_counter_df = org_counter_df.rename(columns={'index': 'ORG', 0: 'Count'})
org_counter_df = org_counter_df.sort_values(by="Count", ascending=False, ignore_index=True)
org_counter_df

,ORG,Count
0,twitter,15
1,facebook,5
2,exo,4
3,eu,4
4,youtube,3
...,...,...
183,makeup,1
184,gatorade,1
185,alabama,1
186,banking,1


In [8]:
test_ner_loc_counter = Counter(combined_test_ner_loc)

print(len(test_ner_loc_counter))

140


In [9]:
loc_counter_df = pd.DataFrame.from_dict(test_ner_loc_counter, orient='index').reset_index()
loc_counter_df = loc_counter_df.rename(columns={'index': 'LOC', 0: 'Count'})
loc_counter_df = loc_counter_df.sort_values(by="Count", ascending=False, ignore_index=True)
loc_counter_df

,LOC,Count
0,chicago,3
1,west,3
2,london,2
3,lake,2
4,us,2
...,...,...
135,indianapolis,1
136,counties,1
137,augustine,1
138,sabine,1


In [10]:
test_ner_misc_counter = Counter(combined_test_ner_misc)

print(len(test_ner_misc_counter))

304


In [11]:
misc_counter_df = pd.DataFrame.from_dict(test_ner_misc_counter, orient='index').reset_index()
misc_counter_df = misc_counter_df.rename(columns={'index': 'MISC', 0: 'Count'})
misc_counter_df = misc_counter_df.sort_values(by="Count", ascending=False, ignore_index=True)
misc_counter_df

,MISC,Count
0,the,12
1,'s,7
2,world,5
3,muslims,4
4,cup,3
...,...,...
299,meathook,1
300,seed,1
301,hamilton,1
302,ayo,1


In [12]:
test_merged_together = pd.concat([per_counter_df, org_counter_df, loc_counter_df, misc_counter_df], axis=1)
test_merged_together

,PER,Count,ORG,Count,LOC,Count,MISC,Count
0,jesus,5,twitter,15.0,chicago,3.0,the,12.0
1,trump,4,facebook,5.0,west,3.0,'s,7.0
2,louis,4,exo,4.0,london,2.0,world,5.0
3,harry,4,eu,4.0,lake,2.0,muslims,4.0
4,donald,4,youtube,3.0,us,2.0,cup,3.0
...,...,...,...,...,...,...,...,...
337,beckham,1,NaN,NaN,NaN,NaN,NaN,NaN
338,david,1,NaN,NaN,NaN,NaN,NaN,NaN
339,trumps,1,NaN,NaN,NaN,NaN,NaN,NaN
340,putin,1,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
test_merged_together.to_csv('logs/test_merged_together.tsv', sep='\t', index=False)